In [124]:
import testjx
import pandas as pd
import numpy as np
from scipy.optimize import fsolve
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.metrics import classification_report
import scipy.stats as st

from sklearn.linear_model import LogisticRegression
import time
from sklearn.metrics import fbeta_score,f1_score,balanced_accuracy_score,recall_score,precision_score
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split
from cvxopt import matrix
from cvxopt import solvers

class Mean_uncertain:
    def __init__(self,model=LogisticRegression(),n=100):
        self.n=n
        self.model=model
        
        
        
    def sigmoid_(self,x,u):
        return 1.0/(1.0+np.exp(-x-u))
    def meanuncertainty(self,x):
        r=[]
        for i in range(0,len(x)+1-self.n,self.n//10):
            r.append(np.mean(x[i:i+self.n]))
        return min(r),max(r)
    def error(self,X,Y):
        self.model.fit(X,Y)
        pre_in=(np.dot(X,self.model.coef_.T)+self.model.intercept_).reshape(X.shape[0],)
        return pre_in
    def equa_m(self,x,X,Y):
        
        train_err=self.error(X,Y)

        ini_err=Y-self.sigmoid_(train_err,x)##ini_err is the predicted err of training set, based on LR

        for k in range(ini_err.shape[0]):
            if Y[k]==1:
                ini_err[k]=0.5*(Y.shape[0]/sum(Y))*ini_err[k]

        return self.meanuncertainty(ini_err)[1]

   
    
    def mean_uncertainty(self,X,Y):
        mean_lr=fsolve(lambda x:self.equa_m(x,X,Y),0.5 )
        return mean_lr
    
    def predict(self,X,Y,x_te,mean_u):
        self.model.fit(X,Y)
        prob_y=self.sigmoid_((np.dot(x_te,self.model.coef_.T)+self.model.intercept_).reshape(x_te.shape[0],),mean_u
             )

        prdict_y=np.round(prob_y)
        return prdict_y

In [128]:
def Meanuncertain_CV(X,y,model=Mean_uncertain()):
    
    bacc,f2,rec,pre=list(),list(),list(),list()
    mean_u=model.mean_uncertainty(X,y)
    for train_index,test_index in cv.split(X,y):
            
            prdict_y=model.predict(X[train_index],y[train_index],X[test_index],mean_u)
            rec.append(recall_score(y[test_index],prdict_y))
            pre.append(precision_score(y[test_index],prdict_y))

            bacc.append(balanced_accuracy_score(y[test_index],prdict_y))
            f2.append(fbeta_score(y[test_index],prdict_y,beta=2))
    return mean_u,bacc,f2,rec,pre


In [3]:
ecoli=pd.read_csv('/Users/lvjingzhe/Desktop/璇/modified_althogram/DRM/data/ecoli_.csv',)
L=[]
for k in ecoli['Class'].values:
    if k in ['0','1','2','3' ]:
        L.append(0)
    else:L.append(1)
ecoli['Class']=L    
E=np.array(ecoli)
np.random.seed(4123)
np.random.shuffle(E)
X_sample=E[:,:-1]/np.max(E[:,:-1],axis=0)
y_sample=E[:,-1]


In [54]:
x_tr, x_te, y_tr, y_te = train_test_split(X_sample,y_sample,test_size = 0.2,
                                                  shuffle = True,
                                                  random_state = 10)

In [133]:
cv=RepeatedStratifiedKFold(n_splits=10, n_repeats=1, random_state=0)
metrics_names=['Balanced_acc','F2_score','Recall','Precision']
all_results_m = []
N_list=np.array(range(50,200,5))
c=0
strat=time.time()

for k in N_list:
    M=Meanuncertain_CV(x_tr,y_tr,model=Mean_uncertain(n=k))
    Lo_score=np.mean(M[1:],axis=1)
    metric_res = {'window':k,'upper_mean': M[0]}
    
    for name, value in zip(metrics_names,Lo_score):
            metric_res[name] = value


    all_results_m.append(metric_res)


eva=pd.DataFrame(all_results_m)
bias=eva[eva.iloc[:,-4]==(eva.iloc[:,-4]).max()]
N_optimal=bias['window'].values[0]
print("optimal window size N by fivefold CV: %.3f "%(bias['window']))

optimal window size N by fivefold CV: 80.000 


In [140]:

M=Mean_uncertain(n=80)
mean_u=M.mean_uncertainty(x_tr,y_tr)
predict_y=M.predict(x_tr,y_tr,x_te,mean_u)
print(" upper_mean : %.3f "%(mean_u))

print(classification_report(y_te,predict_y))
print('>%s: Average G-mean:%.3f ' % ('LR_mean:',np.sqrt(recall_score(y_te,predict_y)*recall_score(y_te,predict_y,pos_label=0))))
print('>%s: Average Balanced_Acc: %.3f ' % ('LR_mean:',balanced_accuracy_score(y_te,predict_y)))
print('>%s: Average Fbeta: %.3f' % ('LR_mean:',fbeta_score(y_te,predict_y,beta=max(2,np.log(y_tr.shape[0]/sum(y_tr)-1)))))
print('>%s: Average Recall: %.3f' % ('LR_mean:',recall_score(y_te,predict_y)))    


 upper_mean : 1.811 
              precision    recall  f1-score   support

           0       0.98      0.90      0.94        61
           1       0.50      0.86      0.63         7

    accuracy                           0.90        68
   macro avg       0.74      0.88      0.79        68
weighted avg       0.93      0.90      0.91        68

>LR_mean:: Average G-mean:0.879 
>LR_mean:: Average Balanced_Acc: 0.879 
>LR_mean:: Average Fbeta: 0.776
>LR_mean:: Average Recall: 0.857
